In [18]:
import cvxopt
# import cvxopt.glpk
from cvxopt import matrix, solvers
%matplotlib inline
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from datetime import timedelta
import datetime
import seaborn as sns
import itertools
from scipy.linalg import block_diag
import time
import os
import pickle
from scipy.misc import comb

In [170]:
profile = np.array([list(np.random.permutation(range(1,11))) for i in range(5)])

In [259]:
profile

array([[ 2,  4,  3,  1,  9,  7,  8,  6,  5, 10],
       [10,  4,  1,  8,  5,  9,  7,  2,  3,  6],
       [ 2,  3,  8, 10,  7,  1,  4,  9,  5,  6],
       [10,  5,  4,  7,  9,  2,  1,  8,  6,  3],
       [ 7,  1,  4,  2,  9,  6,  8, 10,  3,  5]])

In [179]:
stringmatrix = '['
for i in range(profile.shape[0]):
    for j in range(profile.shape[1]):
        if j!=profile.shape[1]-1:
            stringmatrix+=str(profile[i][j])+','
        else:
            stringmatrix+=str(profile[i][j])+';'
stringmatrix=stringmatrix[:-1]
stringmatrix+=']'
print (stringmatrix)

[2,4,3,1,9,7,8,6,5,10;10,4,1,8,5,9,7,2,3,6;2,3,8,10,7,1,4,9,5,6;10,5,4,7,9,2,1,8,6,3;7,1,4,2,9,6,8,10,3,5]


In [155]:
n,m = len(profile), len(profile[0])

In [156]:
n, m

(5, 10)

In [157]:
k=3

In [42]:
# profile = np.array([[1,2,3],[2,3,1],[1,3,2]])
# m=3
# n=3
# k=2

In [158]:
from functools import partial
def Ind(m,k,i,r):
    return (i-1)*(m-k+1)+r+m
IndY = partial(Ind,m,k)
IndY(1,1)

11

In [196]:
profile[0,0]

2

In [160]:
from itertools import combinations
print np.array([list(int(y)+1 for y in x) for x in list(combinations(range(m),2))])

SyntaxError: invalid syntax (<ipython-input-160-5ac2c467ac66>, line 2)

In [247]:
nIneq  = int(n*((m-k+2)*(m-k+3)/2-1)+comb(m,k))
nEq = 1
nVar = int(m+n*(m-k+1)+1)
Aineq = np.zeros((nIneq, nVar))
bineq = np.zeros((nIneq, 1))
Aeq = np.zeros((nEq, nVar))
beq = np.zeros((nEq, 1))
f = np.zeros((nVar, 1))
ctype = 'B'*(nVar-1)+'C'
f[-1] = 1
Aeq[0][0:m] = 1
beq = k
countIneq = 1

for i in range(1,n+1):
    for r in range(1,m-k+2):
        for j in range(1,r+1):
            Aineq[countIneq-1, IndY(i,r)-1] = 1
#             print(IndY(i,r))
#             print(Aineq)
            Aineq[countIneq-1, profile[i-1,j-1]-1] = 1
#             print (Aineq)
            bineq[countIneq-1] = 1
            countIneq = countIneq + 1
        Aineq[countIneq-1, IndY(i,r)-1] = -1
        for j in range(1,r+1):
            Aineq[countIneq-1, profile[i-1,j-1]-1] = -1
        bineq[countIneq-1] = -1
        countIneq = countIneq + 1
list_T = np.array([list(int(y)+1 for y in x) for x in list(combinations(range(m),k))])
for indT in range(1, list_T.shape[0]+1):
    T = list_T[indT-1]
    first_indices_T = np.array([[i for i,elem in enumerate(x) if x[i] in T][0] + 1 for x in profile])
    Aineq[countIneq-1, nVar-1] = -1
    for i in range(1,n+1):
        Aineq[countIneq-1,IndY(i,first_indices_T[i-1])-1] = 1./first_indices_T[i-1]
    countIneq = countIneq + 1

In [248]:
from __future__ import print_function

import sys

import cplex
from cplex.exceptions import CplexError

In [257]:
# data common to all populateby functions
my_obj = list(i[0] for i in f)
my_ub = [cplex.infinity]*len(f)
my_lb = [0.0]*len(f)
my_ctype = ctype
my_colnames = ['x{}'.format(i) for i in range(len(f))]
my_rhs = list(i[0] for i in bineq) + [float(beq)]
my_rownames = ['r{}'.format(i) for i in range(nIneq+nEq)]
my_sense = 'L'*nIneq+'E'*nEq

def populatebynonzero(prob):
    prob.objective.set_sense(prob.objective.sense.minimize)

    prob.linear_constraints.add(rhs=my_rhs, senses=my_sense,
                                names=my_rownames)
    prob.variables.add(obj=my_obj, lb=my_lb, ub=my_ub, types=my_ctype,
                       names=my_colnames)

    rows, cols = np.nonzero(np.concatenate((Aineq,Aeq)))
    rows=list(rows)
    cols=list(cols)
    vals = [np.concatenate((Aineq,Aeq))[rows[i],cols[i]] for i in range(len(rows))]

    prob.linear_constraints.set_coefficients(zip(rows, cols, vals))


def mipex1(pop_method):

    try:
        my_prob = cplex.Cplex()

        handle = populatebynonzero(my_prob)
        
        my_prob.solve()
    except CplexError as exc:
        print(exc)
        return

    print()
    # solution.get_status() returns an integer code
    print("Solution status = ", my_prob.solution.get_status(), ":", end=' ')
    # the following line prints the corresponding string
    print(my_prob.solution.status[my_prob.solution.get_status()])
    print("Solution value  = ", my_prob.solution.get_objective_value())

    numcols = my_prob.variables.get_num()
    numrows = my_prob.linear_constraints.get_num()

    slack = my_prob.solution.get_linear_slacks()
    x = my_prob.solution.get_values()

#     for j in range(numrows):
#         print("Row %d:  Slack = %10f" % (j, slack[j]))
    for j in range(m):
        print("Column %d:  Value = %10f" % (j, x[j]))
    print ("Column %d:  Value = %10f" % (numcols-1, x[numcols-1]))

In [258]:
mipex1('n')

CPXPARAM_Read_DataCheck                          1
CPXPARAM_Read_APIEncoding                        "UTF-8"
CPXPARAM_MIP_Strategy_CallbackReducedLP          0


Tried aggregator 2 times.
MIP Presolve eliminated 35 rows and 0 columns.
Aggregator did 5 substitutions.
Reduced MIP has 301 rows, 46 columns, and 1062 nonzeros.
Reduced MIP has 45 binaries, 0 generals, 0 SOSs, and 0 indicators.
Presolve time = 0.01 sec. (1.24 ticks)
Found incumbent of value 0.000000 after 0.02 sec. (1.39 ticks)

Root node processing (before b&c):
  Real time             =    0.02 sec. (1.40 ticks)
Parallel b&c, 4 threads:
  Real time             =    0.00 sec. (0.00 ticks)
  Sync time (average)   =    0.00 sec.
  Wait time (average)   =    0.00 sec.
                          ------------
Total (root+branch&cut) =    0.02 sec. (1.40 ticks)

Solution status =  101 : MIP_optimal
Solution value  =  0.0
Column 0:  Value =   0.000000
Column 1:  Value =   1.000000
Column 2:  Value =   0.000000
Column 3:  Value =   0.000000
Column 4:  Value =   0.000000
Column 5:  Value =   0.000000
Column 6:  Value =   1.000000
Column 7:  Value =   0.000000
Column 8:  Value =   0.000000
Colu

In [197]:
matlab = np.loadtxt('../nisarg_code/testmatrix', delimiter=',')

In [198]:
print(matlab)

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0. -1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]]


In [226]:
print(Aineq)

[[ 0.  1.  0. ...,  0.  0.  0.]
 [ 0. -1.  0. ...,  0.  0.  0.]
 [ 0.  1.  0. ...,  0.  0.  0.]
 ..., 
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]
 [ 0.  0.  0. ...,  0.  0. -1.]]
